# Xarray with Dask Arrays

In this notebook we show how to apply xarray operations on Dask DataArray objects.

- Open a dataset
- Use standard xarray operations
- Persist data in memory
- Custom workflows and automatic parallelisation

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found [here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). 

-----

- Original Authors: NCI Virtual Research Environment Team
- Keywords: CMIP6, xarray, dask
- CreationDate: 2020-May
---
-  Adapted to DKRZ jupyterhub and data pool: S. Kindermann

In [ ]:
%matplotlib inline

import dask
import xarray as xr

### Start Dask Client for Dashboard


In [ ]:
# set the correct status link for dashboard
import dask
from dask.distributed import Client

dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

In [ ]:
from dask.distributed import Client
client = Client(n_workers=4)
client

Starting the Dask Client will provide a dashboard which is useful to gain insight into the computation. The link to the dashboard will become visible when you create the Client. We recommend having the Client open on one side of your screen and your notebook open on the other side, which will be useful for learning purposes.

### Open a dataset

Xarray is an open source project and Python package that extends the labeled data functionality of Pandas to N-dimensional array-like datasets. It shares a similar API to NumPy and Pandas and supports both Dask and NumPy arrays under the hood. 

By specifying the chunk shape, xarray will automatically create Dask arrays for each data variable in the `Dataset`. In xarray, `Datasets` are dict-like containers of labelled arrays, analogous to the `pandas.DataFrame`. Note that we're taking advantage of xarray's dimension labels when specifying chunk shapes.

In [ ]:
!ls /pool/data/CMIP6/data/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/6hrPlevPt/psl/gn/v20191108/*.nc

In [ ]:
# On DKRZ system
filename = '/pool/data/CMIP6/data/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/6hrPlevPt/psl/gn/v20191108/*.nc'
#filename = '/pool/data/CMIP6/data/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/6hrLevPt/psl/v20191108/gn/latest/*.nc'
ds = xr.open_mfdataset(filename,combine='nested', concat_dim='time',chunks={'lat': 144, 'lon': 92, 'time': -1})
ds = ds.sel(time=slice('1950','1951'))
ds

Quickly inspecting the `Dataset` above, we'll note that this `Dataset` has four _dimensions_ akin to axes in NumPy (`bnds`, `lat`, `lon`, and `time`), three _coordinate variables_ akin to `pandas.Index` objects (also named `lat`, `lon`, and `time`), and one data variable (`psl`) as well as two data varibles defining the lat and lon cell bounds (`lat_bnds`, `lon_bnds`). Xarray also holds Dataset specific metadata as _attributes_.

In [ ]:
da = ds['psl']
da

Each data variable in xarray is called a `DataArray`. These are the fundemental labelled array object in xarray. Much like the `Dataset`, `DataArrays` also have _dimensions_ and _coordinates_ that support many of its label-based opperations.

In [ ]:
da.data

Accessing the underlying array of data is done via the `data` property. Here we can see that the Dask array is backed by a NumPy array of the actual values.

### Use Standard Xarray Operations

In almost all cases, operations using xarray objects are identical regardless of whether the underlying data is stored as a Dask array or a NumPy array.

In [ ]:
da2 = da.groupby('time.month').mean('time')
da3 = da.groupby('time.month') - da2
da3

In [ ]:
da3.data

Call `.compute()` or `.load()` when you want your result as an `xarray.DataArray` with data stored as NumPy arrays.

If you started `Client()` above then you may want to watch the status page during computation.

In [ ]:
computed_da = da3.load()
type(computed_da.data)

In [ ]:
computed_da

### Persist data in memory

If you have the available RAM for your dataset then you can persist data in memory.  

This allows future computations to be much faster.

In [ ]:
da = da.persist()

### Time Series Operations

We want datetime index time-series operations to work efficiently. Here we demo the use of xarray's resample method:

In [ ]:
da.resample(time='1D').mean('time')

Apply a rolling window of 30 timesteps:

In [ ]:
da_smooth = da.rolling(time=30).mean().persist()
da_smooth

Since xarray stores each of its coordinate variables in memory, slicing by label is trivial and entirely lazy.

In [ ]:
%time da.sel(time='1950-01-01T18:00:00')

In [ ]:
%time da.sel(time='1950-01-01T18:00:00').load()

### Custom workflows and automatic parallelisation
Almost all of xarray’s built-in operations work on Dask arrays. If you want to use a function that isn’t wrapped by xarray, one option is to extract Dask arrays from xarray objects (`.data`) and use Dask directly.

Another option is to use xarray’s `apply_ufunc()` function, which can automate embarrassingly parallel “map” type operations where a function written for processing NumPy arrays should be repeatedly applied to xarray objects containing Dask arrays. It works similarly to `dask.array.map_blocks()` and `dask.array.atop()`, but without requiring an intermediate layer of abstraction.

Here we show an example using NumPy operations and a fast function from `bottleneck`, which we use to calculate Spearman’s rank-correlation coefficient:

In [ ]:
import numpy as np
import xarray as xr
import bottleneck

def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)

def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

In the examples above, we were working with some atmospheric pressure data. For this example, we'll calculate the spearman correlation using the raw pressure data with the smoothed version that we also created (`da_smooth`). For this, we'll also have to rechunk the data ahead of time.

In [ ]:
corr = spearman_correlation(da.chunk({'time': -1}),
                            da_smooth.chunk({'time': -1}),
                            'time')
corr

Let's plot this data and add coastlines for clarity.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
f, axs = plt.subplots(1,1,figsize=(12,8))
p = plt.subplot(111, projection=ccrs.PlateCarree())
p = corr.plot(transform=ccrs.PlateCarree())
p.axes.coastlines()

### Close the client

Before moving on to the next exercise, make sure to close your client or stop this kernel.

In [ ]:
client.close()

## Reference

https://examples.dask.org/xarray.html